# Artificial Neural Network

**Artificial neural networks** (ANNs) are computational models inspired by the human brain. 
They are comprised of a large number of connected nodes, each of which performs a simple mathematical operation. 
Each node's output is determined by this operation, as well as a set of parameters that are specific to that node.
By connecting these nodes together and carefully setting their parameters, very complex functions can be learned and calculated.

Artificial neural networks are responsible for many of the recent advances in artificial intelligence, including voice recognition, 
image recognition, and robotics. For example, ANNs can perform image recognition on hand drawn digits.

## Online Learning

With the advent of computers in the 1940s, computer scientists' attention turned towards developing intelligent systems that could learn to perform **prediction and decision making**. In particular online learning,
which is a learning method that can be applied to data points arriving sequentially. This is in opposition to batch learning, which requires that all of the data be present at the time of training.

Online learning is especially useful in scenarios where training data is arriving sequentially over time, such as speech data or the movement of stock prices. With a system capable of online learning, one doesn't 
have to wait until the system has received a ton of data before it can make a prediction or decision. If the human brain learned by batch learning, then human children would take 10 years before they could learn 
to speak, mostly just to gather enough speech data and grammatical rules to speak correctly. Instead, children learn to speak by observing the speech patterns of those around them and gradually incorporating 
that knowledge to improve their own speech, an example of online learning.


## Neurons

Neurons function by firing when they receive enough input from the other neurons to which they're connected. Typically, the output function is modeled as an **activation function**, where inputs below a certain threshold 
don't cause the neuron to fire, and those above the threshold do. Thus, a neuron exhibits what is known as all-or-nothing firing, meaning it is either firing, or it is completely off and no output is produced.

## A Computational Model of the Neuron

As stated above, neurons fire above a certain threshold and do nothing below that threshold, so a model of the neuron requires a function exhibiting the same properties. 
The simplest function that does this is the step function.

The step function is defined as:

\begin{equation*}
H(x) = \left\{
        \begin{array}{ll}
            1 & \quad x \geq 0 \\
            0 & \quad x < 0
        \end{array}
    \right.
\end{equation*}

In this simple neuron model, the input is a single number that must exceed the activation threshold in order to trigger firing. However, neurons can (and should, if they're to do anything useful) have connections 
to multiple incoming neurons, so we need some way of "integrating" these incoming neuron's inputs into a single number. The most common way of doing this is to take a weighted sum of the neuron's incoming inputs, 
so that the neuron fires when the weighted sum exceeds the threshold. To further improve the modeling capacity of the neuron, we want to be able to set the threshold arbitrarily. This can be achieved by adding a 
scalar (which may be positive or negative) to the weighted sum of the inputs.

Thus calculating the output of our neuron model is comprised of two steps:
1. Calculate the sum $w\cdot x + b$ for vectors $w,x$ and scalar $b$, where $x$ is the vector of outputs from the incoming neurons, $w$ is the weighted vector.
2.  Calculate the output, which is the activation function applied to the result of 1. 

Following from the description of step 2, our neuron model defines a **linear classifier**, i.e. a function that splits the inputs into two regions with a linear boundary. 
In two dimensions, this is a line, while in higher dimensions the boundary is known as a hyperplane. The weight vector $w$ 
$w$ defines the slope of the linear boundary while the bias defines the intercept of the linear boundary. The following diagram illustrates a neuron's output for two
incoming connections. Note that the neuron inputs are clearly separated into values of 0 and 1 by a line (defined by $w\cdot x + b = 0$).

![title](img/picture.png)

By adjusting the values of the weights and bias, the step function unit can adjust its linear boundary and learn to split its inputs into classes 0 and 1. Note that 
different values of $w$ and $b$ for multiple step function units will yield multiple different linear classifiers.

## The Universal Approximation Theorem

Since the brain can calculate more than just linear functions by connecting many neurons together, this suggests that connecting many linear classifiers together should produce 
a nonlinear function. In fact, it is proven that for certain activation functions and a very large number of neurons, ANNs can model any continuous, smooth function arbitrarily 
well, a result known as the **universal approximation theorem**.

This is very convenient because, like the brain, an ANN should ideally be able to learn any function handed to it. If ANNs could only learn one type of function (e.g. third degree 
polynomials), this would severely limit the types of problems to which they could be applied. Furthermore, learning often happens in an environment where the type of function to be 
learned is not known beforehand, so it is advantageous to have a model that does not depend on knowing a priori the form of the data it will be exposed to.

Unfortunately, since the step function can only output two different values, 0 and 1, an ANN of step function neurons cannot be a universal approximator (generally speaking, 
continuous functions take on more than two values). Luckily, there is a continuous function called the sigmoid function, described in the next section, that is very similar to 
the step function and can be used in universal approximators.

## The Sigmoid Function

There is a continuous approximation of the step function called the logistic curve, or sigmoid function, denoted as $\sigma(x)$. This function's output ranges over all 
values between 0 and 1 and makes a transition from values near 0 to values near 1 at $x = 0$, similar to the step function $H(x)$.

The sigmoid function is defined as:
$$
\sigma(x) = \frac{1}{1+e^{-x}}
$$

So, for a computational unit that uses the sigmoid function, instead of firing 0 or 1 like a step function unit, it's output will be between 0 and 1, non-inclusive. 
This changes slightly the interpretation of this unit as a model of a neuron, since it no longer exhibits all-or-nothing behavior since it will never take on the value 
of 0 (nothing) or 1 (all). However, the sigmoid function is very close to 0 for $x \lt 0$ and very close to 11 for $x \gt 0$, so it can be interpreted as exhibiting 
practically all-or-nothing behavior on most ($x \not\approx 0$) inputs.

The sigmoid function turns out to be a member of the class of activation functions for universal approximators, so it imitates the behavior of real neurons (by approximating the 
step function) while also permitting the possibility of arbitrary function approximation. In fact, some ANNs use activation functions that are different from the sigmoidal 
function, because those functions are also proven to be in the class of functions for which universal approximators can be built. Two well-known activation functions used in 
the same manner as the **sigmoidal** function are the **hyperbolic tangent** and the **rectifier**.

## Putting It All Together

Neurons are connected to one another, with each neuron's incoming connections made up of the outgoing connections of other neurons. Thus, the ANN will need to connect the 
outputs of sigmoidal units to the inputs of other sigmoidal units.

### One Sigmoidal Unit

The diagram below shows a sigmoidal unit with three inputs $x = (x_1, x_2, x_3)$, one output $y$, bias $b$, and weight vector $w = (w_1, w_2, w_3)$.

![title](img/picture2.png)

### ANNs as Graphs

Artificial neural networks are most easily visualized in terms of a directed graph. In the case of sigmoidal units, node $s$ represents sigmoidal unit $s$ (as in the diagram above) 
and directed edge $e = (u, v)$ indicates that one of sigmoidal unit $v$'s inputs is the output of sigmoidal unit u. 

Thus, if the diagram above represents sigmoidal unit $s$ and inputs $x_1, x_2, x_3$ are the outputs of sigmoidal units A, B, C respectively, then the graph representation of the above
sigmoidal unit will have nodes A, B, C and $s$ with directed edges $(a,s)$, $(b,s)$, and $(c,s)$. Furthermore, since each incoming directed edge is associated with a component of the
weight vector for sigmoidal unit $s$, each incoming edge will be labeled with its corresponding weight component. Thus edge $(a,s)$ will have label $w_1$, $(b,s)$ will have label $w_2$,
and $(c,s)$ will have label $w_3$. The coresponding graph is shown below, with edges feeding into nodes A,B, and C representing inputs to those nodes.

![title](img/picture3.png)

While the above ANN is very simple, ANNs in general can have many more nodes (e.g. modern machine vision applications use ANNs with more than $10^6$ 6 nodes) 
in very complicated connection patterns.

 If the graph above was modified so that's $s$'s output was an input of A, a directed edge passing from $s$ to A would be added, creating what is known as a cycle. This would mean 
 that $s$'s output is dependent on itself. Cyclic computation graphs greatly complicate computation and learning, so computation graphs are commonly restricted to be 
 **directed acyclic graphs** (or DAGs), which have no cycles. ANNs with DAG computation graphs are known as feedforward neural networks, while ANNs with cycles are known as 
 **recurrent neural networks**.


Ultimately, ANNs are used to compute and learn functions. This consists of giving the ANN a series of input-output pairs $(x_i,y_i)$, and training the model to approximate the
function f such that $f(x_i) = y_i$ for all pairs. Thus if $x$ is n-dimensional and $y$ is m-dimensional, the final sigmoidal ANN graph will consist of n input nodes representing
$x = (x_1, x_2, ..., x_n)$ $k$ sigmoidal units and $m$ output nodes representing $y=(y_1, y_2, ..., y_m)$.

An example graph of an ANN computing a two dimensional output $y$ on a three dimensional input $x$ using five sigmoidal units $S_1, S_2, S_3, S_4, S_5$ is shown below. An edge
labeled with weigh $w_{ab}$ represents the component of the weight vector for node $b$ that corresponds to the input coming from node $a$. Note that this is a feedfoward neural network.

![title](img/picture4.png)

Sometimes, output nodes use the same integration and activation as sigmoidal units, while other times they may use more complicated functions, such as the softmax function, 
which is heavily used in classification problems. Often, the choice of integration and activation functions is dependent on the form of the output. For example, since sigmoidal 
units can only output values in the range $(0, 1)$, they are ill-suited to problems where the expected value of $y$ lies outside that range.

### Layers

It is obvious from this computational flow that certain sets of nodes tend to be computed at the same time, since a different set of nodes uses their outputs as inputs. For example
$\{ S_3, S_4, S_5\}$ depends on $\{S_1, S_2\}$. These sets of nodes that are computed together are known as **layers**, and ANNs are generally thought of a series of such layers,
with each layer $l_i$ depending on layer $l_{i-1}$. Thus the above graph is composed of four layers. The first layer $l_0$ is called the **input layer** (which does not need to be
computed, since it is given), while the final layer $l_3$ is called the **output layer**. The intermediate layers are known as **hidden layers** which in this case are the layers
$l_1 = \{ S_3, S_4, S_5\}$ and $l_2 = \{S_1, S_2\}$, are usually numbered so that hidden layer $h_i$ corresponds to layer $l_i$. Thus $h_1 = l_1$ and $h_2 = l_2$. The diagram below 
shows the example ANN with each node grouped into its appropriate layer.

![title](img/picture5.png)

## Training The Model

The ANN can now calculate some function $f_\theta(x)$ that depends on the values of the individual nodes' weight vectors and biases, which together are known as the ANN's parameters
$\theta$. The logical next step is to determine how to alter those biases and weight vectors so that the ANN computes known values of the function. That is, given a series of input-output
pairs $(x_i, y_i)$, how can the weight vectors and biases be altered such that $f_\theta(x_i)\approx y_i$ for all $i$?

### Choosing an Error Function

The typical way to do this is define an error function $E$ over the set of pairs $X = \{ (x_1,y_1), ..., (x_N,y_N) \}$ such that $E(X, \theta)$ is small when 
$f_\theta(x_i) \approx y_i$ for all $i$. Common choices for E are the **mean squared error** (MSE) in the case of regression problems and the **cross entropy** in the case of classification
problems. Thus, training the ANN reduces to minimizing the error $E(X, \theta)$ w.r.t. the parameters (since X is fixed). 

### Gradient Descent

Since the error function $E(X, \theta)$ defines a fairly complex function, finding the minimum analytically is generally impossible. Luckly, there exists a general method for
minimizing differentiable functions called **gradient descent**. Learning for an ANN typically starts with a random initialization of teh parameters (the weight vectors and biases) followed
by successive updates to those parameters based on gradient descent until the error function converges.

A major advantage of gradient descent is that it can be used for online learning, since the parameters are not solved in one calculation but are instead gradually improved by 
moving in the direction of the negative gradient. Thus, if input-output pairs are arriving in a sequential fashion, the ANN can perform gradient descent on one input-output 
pair for a certain number of steps, and then do the same once the next input-output pair arrives. For an appropriate choice of step size, this approach can yield results 
similar to gradient descent on the entire dataset $X$ (known as **batch learning**).

Because gradient descent is a local method (the step direction is determined by the gradient at a single point), it can only find local minima. While this is generally a 
significant problem for most optimization applications, recent research has suggested that finding local minima is not actually an issue for ANNs, since the vast majority of 
local minima are evenly distributed and similar in magnitude for large ANNs.

### Backpropagation

For a long time, calculating the gradient for ANNs was thought to be mathematically intractable, since ANNs can have large numbers of nodes and very many layers, making the error 
function $E(X, \theta)$ highly nonlinear. However, in the mid-1980s, computer scientists were able to derive a method for calculating the gradient with respect to an ANN's 
parameters, known as **backpropagation**, or "backpropagation by errors". The method works for both feedforward neural networks (for which it was originally designed) as well as 
for recurrent neural networks, in which case it is called **backpropagation through time**, or BPTT. The discovery of this method brought about a renaissance in artificial neural 
network research, as training non-trivial ANNs had finally become feasible.


